<a href="https://colab.research.google.com/github/Swathi04/CMPE255_Approximate_Nearest_Neighbour/blob/main/AssignmentCMPE255ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Using state of art libraries for Approximate nearest neighbor search
Submitted by: SWATHI ANANDRAM





##INSTALLING NECESSARY LIBRARIES

In [1]:
!pip install lightfm
!pip install nmslib
!pip install faiss
!pip install annoy


     |████████████████████████████████| 310 kB 27.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705346 sha256=3b6ac9edbd009278118b0b3bfb94fd090abb2f546ff751a1a522fc6424153a73
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
     |████████████████████████████████| 13.5 MB 29.0 MB/s 
     |████████████████████████████████| 188 kB 43.7 MB/s 
     |████████████████████████████████| 4.7 MB 28.7 MB/s 
     |████████████████████████████████| 646 kB 29.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391650 sha256=1fade6e3c1b41419fdd12b4a966a3d80010071343c3bb7a4794a61f4db18c65a
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [3]:
!pip install faiss-cpu --no-cache

     |████████████████████████████████| 8.4 MB 33.2 MB/s 


##IMPORTING NECESSARY LIBRARIES

In [89]:
import pickle
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import nmslib
import faiss
import annoy
import time
from stackapi import StackAPI

##INSTALL STACKAPI TO GET THE DATA

In [5]:
!pip install stackapi

  Created wheel for stackapi: filename=StackAPI-0.2.0-py3-none-any.whl size=5856 sha256=542b93b2adb8b28ee85c6f8fbe122ef559ee8b73e6dac85f4bc9515f68d98a5f
  Stored in directory: /root/.cache/pip/wheels/ec/db/60/df42a65853e3581c26a2fbb2012a228cb8e267369a3b9ca44d
Successfully built stackapi


##LOAD DATA

In [6]:
stackexchange = fetch_stackexchange('crossvalidated')
stackexchange

{'item_feature_labels': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'item_features': <72360x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 72360 stored elements in Compressed Sparse Row format>,
 'test': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 8020 stored elements in COOrdinate format>,
 'train': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 51477 stored elements in COOrdinate format>}

In [9]:
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

In [10]:
stackexchange['item_feature_labels']

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

###Pickle data

In [11]:
with open('stack_exchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

###Load data and visualize it

In [12]:
def load_data():
    with open('stack_exchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[-0.08981958,  0.01580326,  0.04165578, ...,  0.01822499,
         -0.03202373,  0.11310798],
        [ 0.174731  , -0.03626881,  0.17524493, ..., -0.0752098 ,
          0.00059961,  0.10559047],
        [ 0.02355818,  0.13794656,  0.1261663 , ..., -0.00232866,
          0.08617596,  0.0461015 ],
        ...,
        [-0.00544038,  0.00064837,  0.00693827, ...,  0.02580006,
          0.0055762 , -0.01024269],
        [-0.08661043,  0.05814417, -0.05294326, ..., -0.08642083,
          0.02310379, -0.10750322],
        [-0.00310272,  0.01127101, -0.0112341 , ..., -0.05971843,
          0.00136822,  0.02068056]], dtype=float32)}

##LSH
Locality-sensitive hashing (LSH) is an algorithmic technique that hashes similar input items into the same "buckets" with high probability. The number of buckets is much smaller than the universe of possible input items. Since similar items end up in the same buckets, this technique can be used for data clustering and nearest neighbor search. It differs from conventional hashing techniques in that hash collisions are maximized, not minimized. 

###VISUALIZING VECTORS

In [13]:
vectors_value = data["vector"]
names_value = data["name"]

In [14]:
vectors_value

array([[-0.08981958,  0.01580326,  0.04165578, ...,  0.01822499,
        -0.03202373,  0.11310798],
       [ 0.174731  , -0.03626881,  0.17524493, ..., -0.0752098 ,
         0.00059961,  0.10559047],
       [ 0.02355818,  0.13794656,  0.1261663 , ..., -0.00232866,
         0.08617596,  0.0461015 ],
       ...,
       [-0.00544038,  0.00064837,  0.00693827, ...,  0.02580006,
         0.0055762 , -0.01024269],
       [-0.08661043,  0.05814417, -0.05294326, ..., -0.08642083,
         0.02310379, -0.10750322],
       [-0.00310272,  0.01127101, -0.0112341 , ..., -0.05971843,
         0.00136822,  0.02068056]], dtype=float32)

In [15]:
names_value

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

###Creating Index Class

In [43]:
class LSHIndex():
  def __init__(self, vectors, labels):
      self.dimension = vectors.shape[1]
      self.vectors = vectors.astype('float32')
      self.labels = labels
  def build_index(self, num_bits=8):
      self.index = faiss.IndexLSH(self.dimension, num_bits)
      self.index.add(self.vectors)
  def results(self, vectors, k=10):
      distances, indices = self.index.search(vectors, k) 
      return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [95]:
time_1 = time.time()

lsh_ind = LSHIndex(data["vector"], data["name"])
lsh_ind.build_index()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(lsh_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("LSH took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:779
 question_id:1063
 question_id:786
 question_id:731
 question_id:6
 question_id:1296
 question_id:1384
 question_id:1517
 question_id:770
 question_id:337
LSH took 0.019115924835205078 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [106]:
site_name = 'stackoverflow'
site_value = StackAPI(site_name)
question = site_value.fetch('questions/{ids}', ids=[337], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'items': [{'accepted_answer_id': 342,
   'answer_count': 12,
   'body': "<p>I am about to build a piece of a project that will need to construct and post an XML document to a web service and I'd like to do it in Python, as a means to expand my skills in it.  </p>\n\n<p>Unfortunately, whilst I know the XML model fairly well in .NET, I'm uncertain what the pros and cons are of the XML models in Python.  </p>\n\n<p>Anyone have experience doing XML processing in Python? Where would you suggest I start? The XML files I'll be building will be fairly simple.</p>\n",
   'content_license': 'CC BY-SA 3.0',
   'creation_date': 1217648155,
   'is_answered': True,
   'last_activity_date': 1636743381,
   'last_edit_date': 1483102581,
   'link': 'https://stackoverflow.com/questions/337/xml-processing-in-python',
   'owner': {'accept_rate': 73,
    'display_name': 'saalon',
    'link': 'https://stackoverflow.com/users/111/saalon',
    'profile_image': 'https://www.g

Question with question id - 6

In [97]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Question with question id - 1010

In [107]:
print("Following question is similar:\n" + question['items'][0]['title'])


Following question is similar:
XML Processing in Python


##Exhaustive Search
Any search process in which every item of a set is checked before a decision is made about the presence or absence of a target item. This may be a search for particular items in memory, a visual search, or any problem-solving exercise that involves choosing the best path to a solution from a number of alternative paths.

###Creating Index Class

In [45]:
class ExhaustiveSearchIndex():
  def __init__(self, vectors, labels):
    self.dimension = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build_index(self):
    self.index = faiss.IndexFlatL2(self.dimension,)
    self.index.add(self.vectors)

  def results(self, vectors, k=10):
    distances, indices = self.index.search(vectors, k) 
    return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [108]:
time_1 = time.time()

exi_ind = ExhaustiveSearchIndex(data["vector"], data["name"])
exi_ind.build_index()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(exi_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Exhaustive Search took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:9
 question_id:138
 question_id:833
 question_id:28568
 question_id:6744
 question_id:10135
 question_id:555
 question_id:10706
 question_id:338
Exhaustive Search took 0.021610736846923828 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [109]:
question = site_value.fetch('questions/{ids}', ids=[9], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'items': [{'accepted_answer_id': 1404,
   'answer_count': 68,
   'body': "<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years?</p>\n",
   'community_owned_date': 1313523643,
   'content_license': 'CC BY-SA 4.0',
   'creation_date': 1217547659,
   'is_answered': True,
   'last_activity_date': 1637011985,
   'last_edit_date': 1609868012,
   'link': 'https://stackoverflow.com/questions/9/how-do-i-calculate-someones-age-based-on-a-datetime-type-birthday',
   'owner': {'accept_rate': 100,
    'display_name': 'Jeff Atwood',
    'link': 'https://stackoverflow.com/users/1/jeff-atwood',
    'profile_image': 'https://www.gravatar.com/avatar/51d623f33f8b83095db84ff35e15dbe8?s=256&d=identicon&r=PG',
    'reputation': 61685,
    'user_id': 1,
    'user_type': 'registered'},
   'question_id': 9,
   'score': 2068,
   'tags': ['c#', '.net', 'datetime'],
   'title': 'How do I calculate someone&#39;s age based on a 

Question for question id 6

In [111]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


In [113]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
How do I calculate someone&#39;s age based on a DateTime type birthday?


##Trees and Graphs

###Create Index Class

In [64]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def results(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [114]:
time_1 = time.time()

annoy_ind = AnnoyIndex(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][6], data['name'][6]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Trees and Graphs took {total_time} seconds.".format(total_time=total_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:138
 question_id:780
 question_id:443
 question_id:555
 question_id:666
 question_id:338
 question_id:11001
 question_id:10135
 question_id:19753
Trees and Graphs took 0.8037028312683105 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [131]:
question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'items': [{'accepted_answer_id': 31,
   'answer_count': 7,
   'body': "<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <code>percentage-based width</code> on the child <code>div</code>, it collapses to <code>0 width</code> on IE7, but not on Firefox or Safari.</p>\n<p>If I use <code>pixel width</code>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <code>pixel-based width</code> on the child?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",
   'content_license': 'CC BY-SA 4.0',
   'creation_date': 1217542088,
   'is_answered': True,
   'last_activity_date': 1611946005,
   'last_edit_date': 1611946005,
   'link': 'https://stackoverflow.com/questions/6/why-did-the-widt

Question for question id 6

In [153]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Similar question

In [68]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


##Product Quantization
Product quantization (PQ) is an effective vector quantization method. A product quantizer can generate an exponentially large codebook at very low memory/time cost. The essence of PQ is to decompose the high-dimensional vector space into the Cartesian product of subspaces and then quantize these subspaces separately.

###Create Index Class

In [72]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
        
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def results(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [133]:
time_1 = time.time()

annoy_ind = PQIndex(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Product Quantization took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:9
 question_id:1234
 question_id:15
 question_id:778
 question_id:145
 question_id:84
 question_id:20
 question_id:135
 question_id:82
Product Quantization took 3.4982359409332275 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [144]:
question = site_value.fetch('questions/{ids}', ids=[145], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'items': [{'answer_count': 9,
   'body': '<p>Does anyone know of a good way to compress or decompress files and folders in C# quickly? Handling large files might be necessary.</p>\n',
   'community_owned_date': 1396616569,
   'content_license': 'CC BY-SA 4.0',
   'creation_date': 1217610788,
   'is_answered': True,
   'last_activity_date': 1584530660,
   'last_edit_date': 1545747183,
   'link': 'https://stackoverflow.com/questions/145/compressing-decompressing-folders-files',
   'owner': {'accept_rate': 67,
    'display_name': 'DylanJ',
    'link': 'https://stackoverflow.com/users/87/dylanj',
    'profile_image': 'https://www.gravatar.com/avatar/1f8377ba373028db1c2598a963c9ee2c?s=256&d=identicon&r=PG',
    'reputation': 2165,
    'user_id': 87,
    'user_type': 'registered'},
   'question_id': 145,
   'score': 57,
   'tags': ['c#', '.net', 'file', 'compression'],
   'title': 'Compressing / Decompressing Folders &amp; Files',
   'view_count': 5040}],


Question with question id 6

In [138]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


In [139]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
Compressing / Decompressing Folders &amp; Files


##HNSW
Hierarchical Navigable Small World (HNSW) graphs are among the top-performing indexes for vector similarity search[1]. HNSW is a hugely popular technology that time and time again produces state-of-the-art performance with super fast search speeds and fantastic recall.

Yet despite being a popular and robust algorithm for approximate nearest neighbors (ANN) searches, understanding how it works is far from easy.

###Create Class Index

In [147]:
class NMSLIBIndex():
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
        
  def results(self, vector, k=10):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [146]:
time_1 = time.time()

hnsw_ind = NMSLIBIndex(data["vector"], data["name"])
hnsw_ind.build()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(hnsw_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("HNSW took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:9
 question_id:138
 question_id:22
 question_id:833
 question_id:5
 question_id:50
 question_id:131
 question_id:28
 question_id:90
HNSW took 50.4233021736145 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [150]:
question = site_value.fetch('questions/{ids}', ids=[90], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'items': [{'accepted_answer_id': 1466832,
   'answer_count': 3,
   'body': '<p>How do you <a href="http://svnbook.red-bean.com/en/1.8/svn.branchmerge.html" rel="noreferrer">branch and merge</a> with Apache Subversion using the TortoiseSVN client? </p>\n',
   'closed_date': 1356493549,
   'closed_reason': 'not a real question',
   'creation_date': 1217601684,
   'is_answered': True,
   'last_activity_date': 1503594083,
   'last_edit_date': 1503594083,
   'link': 'https://stackoverflow.com/questions/90/how-do-you-branch-and-merge-with-tortoisesvn',
   'owner': {'accept_rate': 91,
    'display_name': 'cmcculloh',
    'link': 'https://stackoverflow.com/users/58/cmcculloh',
    'profile_image': 'https://www.gravatar.com/avatar/298e0497aa6b76a573f17e6a2bb22dec?s=256&d=identicon&r=PG',
    'reputation': 44671,
    'user_id': 58,
    'user_type': 'registered'},
   'question_id': 90,
   'score': 163,
   'tags': ['svn', 'tortoisesvn', 'branch', 'branching-and-

Question id - 6

In [151]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Similar Question

In [152]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
How do you branch and merge with TortoiseSVN?
